# Solar Composition

Here we show how to work with the {py:obj}`LoddersComposition <pynucastro.networks.rate_collection.LoddersComposition>` class, which initializes a composition from the present day solar abundances of {cite:t}`lodders:2020`. To access the data tables, follow this [link](https://sites.wustl.edu/planetarychemistrylaboratory/data-tables/). This class gives the option to set a desired metallicity `Z=0.0` and scales the Lodders abundances accordingly. If no metallicity is provided, it will be calculated from the Lodders data as `(1 - H - He)`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from ipywidgets import interact, FloatSlider

In [ ]:
from pynucastro import networks
from pynucastro.nucdata import Nucleus

## Setting solar composition and finding the top 10 most abundant isotopes

In [ ]:
def top_10_isotopes(comp, n=10):
    fracs = sorted(comp.items(), key=lambda frac: frac[1], reverse=True)
    return fracs[:n]

solar = networks.LoddersComposition() ## Default metallicity
top = top_10_isotopes(solar, n=10)

for nuc, X in top:
    print(f"{nuc.short_spec_name:>6} X= {X:.4f}")

## Binning Lodders composition to a desired network
If we wish to use a network that does not contain all isotopes that are in the Lodders composition, then we use {py:meth}`Composition.bin_as <pynucastro.networks.rate_collection.Composition.bin_as>` which will bin the Lodders data to the desired network: 

In [ ]:
solar = networks.LoddersComposition()

new_comp = [Nucleus("p"),
           Nucleus("he4"),
           Nucleus("o16"),
           Nucleus("c12"),
           Nucleus("ne20"),
           Nucleus("fe56"),
           Nucleus("n14"),
           Nucleus("si28"),
           Nucleus("mg24"),
           Nucleus("s32")] ## Desired network

binned_comp = solar.bin_as(new_comp)
print("Sum of binned comp:", sum(binned_comp.values()))

fig = binned_comp.plot()


## Scaling H and He with desired metallicty Z

In [ ]:
def xyz_from_comp(comp):
    X = 0.0
    Y = 0.0
    Z = 0.0

    for nuc, X_i in comp.items():
        if nuc.Z == 1: # Total H
            X += X_i
        elif nuc.Z == 2: # Total He
            Y += X_i
        else:
            Z += X_i # Total metals

    return X, Y, Z

solar = networks.LoddersComposition()
X_solar, Y_solar, Z_solar = xyz_from_comp(solar)

print("Default solar metalicity is Z = ", Z_solar)

def plot_lodders(Z):

    comp = networks.LoddersComposition(Z=Z)

    X, Y , Z = xyz_from_comp(comp)

    plt.figure(figsize=(4,4))

    labels = ["H", "He", "Metals"]
    values = [X,Y,Z]

    x = np.arange(len(labels))

    fig, ax = plt.subplots()

    ax.bar(x, values)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_ylabel("Mass Fractions")
    ax.set_ylim(0, 1.0)
    ax.set_title(f"Lodders Composition at Z = {Z:.3f}")

    plt.show()

interact(
    plot_lodders,
    Z=FloatSlider(
        value=Z_solar,
        min=0.01,
        max=0.04,
        step=0.001,
        description="Z",
        readout_format=".3f"
    )
)